In [ ]:
import pandas as pd
import numpy as np
from numpy.lib.stride_tricks import sliding_window_view
import os, pickle
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans, DBSCAN, SpectralClustering, AgglomerativeClustering, MiniBatchKMeans, Birch, OPTICS
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
import seaborn as sns

from itertools import product

import sys, os

import trajectory as T                      # trajectory generation
import optimizer as O                       # stochastic gradient descent optimizer
import solver as S                          # MDP solver (value-iteration)
import plot as P

In [ ]:
import pickle
import pandas as pd

with open("X_df_pneumonia.pkl", 'rb') as file:
    X_df = pickle.load(file)

with open("y_df_pneumonia.pkl", 'rb') as file:
    y_df = pickle.load(file)

with open("data_df_pneumonia.pkl", 'rb') as file:
    data_df = pickle.load(file)

with open("data_non_normalized_df_pneumonia.pkl", 'rb') as file:
    data_non_normalized_df = pickle.load(file)

In [ ]:
num_clusters = 100
mbk = MiniBatchKMeans(n_clusters= num_clusters, random_state=0)
mbk.fit(X_df)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


MiniBatchKMeans(n_clusters=100, random_state=0)

In [ ]:
kmeans = KMeans(n_clusters= num_clusters, random_state=0)
kmeans.fit(X_df)
b = Birch(n_clusters=num_clusters, threshold=2)
b.fit(X_df)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Birch(n_clusters=100, threshold=2)

In [ ]:
np.unique(mbk.labels_, return_counts = True)[1]

array([ 566,  233,  193, 1348,    3,  274,  657,   87,  565,  198, 1313,
       1414,  739,  225,  159,  715,  434, 1405,  290,  643,  126,  256,
        614,  635,  115,  168,   24,  116,  520,  987,   76,  252,  684,
        490,  280,  913,  649,  275,   20,  115,  120,  371,  303,  811,
        405,  500,  420,   90,  495,  266,  286,  169,  251,  424,  259,
        179,  539,  347,  208,  167,  871,  392,  148,  184,  557,  637,
         81,  975,  500,  189,  638,  308,  304,  268, 7326,   80,  254,
        248,  403,  278,  144,  135,   88,  219,  345,  435,  624,  621,
        119, 2168,  397,  925,  147,  212,  599,  133, 2285, 1637, 1579,
         70])

In [ ]:
np.unique(kmeans.labels_, return_counts = True)[1]

array([ 649, 7815,  161, 2438,  603,  435, 1259, 2400, 1263,   28, 1579,
        254, 1238,  434,  303,  657,  725,  500,  655,  493,  530,  505,
       2028,   96,  925,  290, 2168,  135,  937, 1859,  987,  110,  393,
        599,  238,  147,  274,  165,  225,  124,  397,   21,  200,   73,
        638,  300,  831,  643,  117,  500,  278,  196,  509,  251,  146,
         64,  266,  822,  169,  539,  135,  399,  424,  248,  378,  557,
         94,  148,  179,   81,  189,  343,  110,  248,   72,  133,  468,
        144,  556,  594,  120,  403,  219,  614,  107,  315,   44,   56,
        119,   42,  112,  392,  156,  284,   81,  142,   87,  289,  180,
         63])

In [ ]:
np.unique(b.labels_, return_counts = True)[1]

array([ 645,  120,  945,  619,  885,  770,  557,  826,  649, 1265,   60,
        505,  133, 1962,  383,   18,   90,  169,  289,  148,  724, 1023,
        315,  684,  276,  124,  500,  987, 1259,  455, 2400,  420,  204,
       1130,   42,   16,  212,  265,   80,  657,   40,  146,  300,  468,
        490,  378,   57,  115,  120,   88,   72, 2168,  248,  196,  251,
        303,  387,  278,   87, 7815,  261,  182,   56,   15,  212, 1238,
        457,  110,  147,  189,  725,  424,  811,  219,   44,  107,  144,
         59, 2028,  509,  434,  594, 2392, 1579,   36,   21,  156,   40,
        254,   50,  110,  216,  248,  556,  135,  144,   78,  530,  238,
        343])

In [ ]:
X_df['k_cluster'] = kmeans.labels_.copy()
data_df['k_cluster'] = kmeans.labels_.copy()
data_non_normalized_df['k_cluster'] = kmeans.labels_.copy()

X_df['m_cluster'] = mbk.labels_.copy()
data_df['m_cluster'] = mbk.labels_.copy()
data_non_normalized_df['m_cluster'] = mbk.labels_.copy()

X_df['b_cluster'] = b.labels_.copy()
data_df['b_cluster'] = b.labels_.copy()
data_non_normalized_df['b_cluster'] = b.labels_.copy()

In [ ]:
clusters = {}
for i in range(100):
  cluster = i
  data = data_non_normalized_df[(data_non_normalized_df['m_cluster'] == i)]
  clusters[i] = data.mean().to_dict()
  clusters[i]['index'] = i

m_full_dict_consolidated = []
for i in clusters:
  index = clusters[i]['index']
  for key, value in clusters[i].items():
    m_full_dict_consolidated.append({'index': index, 'feature': key, "mean": value})

In [ ]:
clusters = {}
for i in range(100):
  cluster = i
  data = data_non_normalized_df[(data_non_normalized_df['k_cluster'] == i)]
  clusters[i] = data.mean().to_dict()
  clusters[i]['index'] = i

k_full_dict_consolidated = []
for i in clusters:
  index = clusters[i]['index']
  for key, value in clusters[i].items():
    k_full_dict_consolidated.append({'index': index, 'feature': key, "mean": value})

In [ ]:
clusters = {}
for i in range(100):
  cluster = i
  data = data_non_normalized_df[(data_non_normalized_df['b_cluster'] == i)]
  clusters[i] = data.mean().to_dict()
  clusters[i]['index'] = i

b_full_dict_consolidated = []
for i in clusters:
  index = clusters[i]['index']
  for key, value in clusters[i].items():
    b_full_dict_consolidated.append({'index': index, 'feature': key, "mean": value})

In [ ]:
m_df = pd.DataFrame(m_full_dict_consolidated)
m_pivoted_df = m_df.pivot(index='feature', columns='index', values='mean')

k_df = pd.DataFrame(k_full_dict_consolidated)
k_pivoted_df = m_df.pivot(index='feature', columns='index', values='mean')

b_df = pd.DataFrame(b_full_dict_consolidated)
b_pivoted_df = m_df.pivot(index='feature', columns='index', values='mean')

In [ ]:
m_new_df = m_pivoted_df.T
m_updated_new_df = m_new_df.drop(['m_cluster', 'k_cluster', 'b_cluster'], axis=1)
csv_filename = 'm_cluster_summary.csv'
m_updated_new_df.to_csv(csv_filename, index=False)

k_new_df = k_pivoted_df.T
k_updated_new_df = k_new_df.drop(['m_cluster', 'k_cluster', 'b_cluster'], axis=1)
csv_filename = 'k_cluster_summary.csv'
k_updated_new_df.to_csv(csv_filename, index=False)

b_new_df = b_pivoted_df.T
b_updated_new_df = b_new_df.drop(['m_cluster', 'k_cluster', 'b_cluster'], axis=1)
csv_filename = 'b_cluster_summary.csv'
b_updated_new_df.to_csv(csv_filename, index=False)

In [ ]:
unique_stay_ids = data_df.index.get_level_values('stay_id').unique()

trajectories = []


for stay_id in unique_stay_ids:


  states, actions = data_df.loc[stay_id]['m_cluster'], data_df.loc[stay_id]['action']

  trajectory = []
  for i in range(len(states) - 1):
    trajectory.append((states[i], int(actions[i]), states[i+1] ))

  if len(trajectory) > 0:
    trajectories.append(T.Trajectory(trajectory))

In [ ]:
terminal_states = []

for traj in trajectories:
  terminal_states.append(traj._t[-1][-1])

terminal_states = list(set(terminal_states))

In [ ]:
"""
Maximum Entropy Inverse Reinforcement Learning and Maximum Causal Entropy
Inverse Reinforcement Learning.

Based on the corresponding paper by B. Ziebart et al. (2008) and the Thesis
by Ziebart (2010).
"""

import numpy as np
from itertools import product


# -- common functions ----------------------------------------------------------

def feature_expectation_from_trajectories(features, trajectories):
    """
    Compute the feature expectation of the given trajectories.

    Simply counts the number of visitations to each feature-instance and
    divides them by the number of trajectories.

    Args:
        features: The feature-matrix (e.g. as numpy array), mapping states
            to features, i.e. a matrix of shape (n_states x n_features).
        trajectories: A list or iterator of `Trajectory` instances.

    Returns:
        The feature-expectation of the provided trajectories as map
        `[state: Integer] -> feature_expectation: Float`.
    """
    n_states, n_features = features.shape

    fe = np.zeros(n_features)

    for t in trajectories:
        for s in t.states():
            fe += features[s, :]

    return fe / len(trajectories)


def initial_probabilities_from_trajectories(n_states, trajectories):
    """
    Compute the probability of a state being a starting state using the
    given trajectories.

    Args:
        n_states: The number of states.
        trajectories: A list or iterator of `Trajectory` instances.

    Returns:
        The probability of a state being a starting-state as map
        `[state: Integer] -> probability: Float`.
    """
    p = np.zeros(n_states)

    for t in trajectories:
        p[t.transitions()[0][0]] += 1.0

    return p / len(trajectories)


def expected_svf_from_policy(p_transition, p_initial, terminal, p_action, eps=1e-5):
    """
    Compute the expected state visitation frequency using the given local
    action probabilities.

    This is the forward pass of Algorithm 1 of the Maximum Entropy IRL paper
    by Ziebart et al. (2008). Alternatively, it can also be found as
    Algorithm 9.3 in in Ziebart's thesis (2010).

    It has been slightly adapted for convergence, by forcing transition
    probabilities from terminal stats to be zero.

    Args:
        p_transition: The transition probabilities of the MDP as table
            `[from: Integer, to: Integer, action: Integer] -> probability: Float`
            specifying the probability of a transition from state `from` to
            state `to` via action `action` to succeed.
        p_initial: The probability of a state being an initial state as map
            `[state: Integer] -> probability: Float`.
        terminal: A list of terminal states.
        p_action: Local action probabilities as map
            `[state: Integer, action: Integer] -> probability: Float`
            as returned by `local_action_probabilities`.
        eps: The threshold to be used as convergence criterion. Convergence
            is assumed if the expected state visitation frequency changes
            less than the threshold on all states in a single iteration.

    Returns:
        The expected state visitation frequencies as map
        `[state: Integer] -> svf: Float`.
    """
    n_states, _, n_actions = p_transition.shape

    # 'fix' our transition probabilities to allow for convergence
    # we will _never_ leave any terminal state
    p_transition = np.copy(p_transition)
    p_transition[terminal, :, :] = 0.0

    # set-up transition matrices for each action
    p_transition = [np.array(p_transition[:, :, a]) for a in range(n_actions)]

    # actual forward-computation of state expectations
    d = np.zeros(n_states)

    delta = np.inf
    while delta > eps:
        d_ = [p_transition[a].T.dot(p_action[:, a] * d) for a in range(n_actions)]
        d_ = p_initial + np.array(d_).sum(axis=0)

        delta, d = np.max(np.abs(d_ - d)), d_

    return d


# -- plain maximum entropy (Ziebart et al. 2008) -------------------------------

def local_action_probabilities(p_transition, terminal, reward):
    """
    Compute the local action probabilities (policy) required for the edge
    frequency calculation for maximum entropy reinfocement learning.

    This is the backward pass of Algorithm 1 of the Maximum Entropy IRL
    paper by Ziebart et al. (2008).

    Args:
        p_transition: The transition probabilities of the MDP as table
            `[from: Integer, to: Integer, action: Integer] -> probability: Float`
            specifying the probability of a transition from state `from` to
            state `to` via action `action` to succeed.
        terminal: A set/list of terminal states.
        reward: The reward signal per state as table
            `[state: Integer] -> reward: Float`.

    Returns:
        The local action probabilities (policy) as map
        `[state: Integer, action: Integer] -> probability: Float`
    """
    n_states, _, n_actions = p_transition.shape

    er = np.exp(reward)
    p = [np.array(p_transition[:, :, a]) for a in range(n_actions)]

    # initialize at terminal states
    zs = np.zeros(n_states)
    zs[terminal] = 1.0

    # perform backward pass
    # This does not converge, instead we iterate a fixed number of steps. The
    # number of steps is chosen to reflect the maximum steps required to
    # guarantee propagation from any state to any other state and back in an
    # arbitrary MDP defined by p_transition.
    for _ in range(2 * n_states):
        za = np.array([er * p[a].dot(zs) for a in range(n_actions)]).T
        zs = za.sum(axis=1)

    # compute local action probabilities
    return za / zs[:, None]


def compute_expected_svf(p_transition, p_initial, terminal, reward, eps=1e-5):
    """
    Compute the expected state visitation frequency for maximum entropy IRL.

    This is an implementation of Algorithm 1 of the Maximum Entropy IRL
    paper by Ziebart et al. (2008).

    This function combines the backward pass implemented in
    `local_action_probabilities` with the forward pass implemented in
    `expected_svf_from_policy`.

    Args:
        p_transition: The transition probabilities of the MDP as table
            `[from: Integer, to: Integer, action: Integer] -> probability: Float`
            specifying the probability of a transition from state `from` to
            state `to` via action `action` to succeed.
        p_initial: The probability of a state being an initial state as map
            `[state: Integer] -> probability: Float`.
        terminal: A list of terminal states.
        reward: The reward signal per state as table
            `[state: Integer] -> reward: Float`.
        eps: The threshold to be used as convergence criterion for the
            expected state-visitation frequency. Convergence is assumed if
            the expected state visitation frequency changes less than the
            threshold on all states in a single iteration.

    Returns:
        The expected state visitation frequencies as map
        `[state: Integer] -> svf: Float`.
    """
    p_action = local_action_probabilities(p_transition, terminal, reward)
    return expected_svf_from_policy(p_transition, p_initial, terminal, p_action, eps)


def irl(p_transition, features, terminal, trajectories, optim, init, eps=1e-4, eps_esvf=1e-5):
    """
    Compute the reward signal given the demonstration trajectories using the
    maximum entropy inverse reinforcement learning algorithm proposed in the
    corresponding paper by Ziebart et al. (2008).

    Args:
        p_transition: The transition probabilities of the MDP as table
            `[from: Integer, to: Integer, action: Integer] -> probability: Float`
            specifying the probability of a transition from state `from` to
            state `to` via action `action` to succeed.
        features: The feature-matrix (e.g. as numpy array), mapping states
            to features, i.e. a matrix of shape (n_states x n_features).
        terminal: A list of terminal states.
        trajectories: A list of `Trajectory` instances representing the
            expert demonstrations.
        optim: The `Optimizer` instance to use for gradient-based
            optimization.
        init: The `Initializer` to use for initialization of the reward
            function parameters.
        eps: The threshold to be used as convergence criterion for the
            reward parameters. Convergence is assumed if all changes in the
            scalar parameters are less than the threshold in a single
            iteration.
        eps_svf: The threshold to be used as convergence criterion for the
            expected state-visitation frequency. Convergence is assumed if
            the expected state visitation frequency changes less than the
            threshold on all states in a single iteration.

    Returns:
        The reward per state as table `[state: Integer] -> reward: Float`.
    """
    n_states, _, n_actions = p_transition.shape
    _, n_features = features.shape

    # compute static properties from trajectories
    e_features = feature_expectation_from_trajectories(features, trajectories)
    p_initial = initial_probabilities_from_trajectories(n_states, trajectories)

    # basic gradient descent
    theta = init(n_features)
    delta = np.inf

    optim.reset(theta)
    while delta > eps:
        theta_old = theta.copy()

        # compute per-state reward
        reward = features.dot(theta)

        # compute the gradient
        e_svf = compute_expected_svf(p_transition, p_initial, terminal, reward, eps_esvf)
        grad = e_features - features.T.dot(e_svf)

        # perform optimization step and compute delta for convergence
        optim.step(grad)
        delta = np.max(np.abs(theta_old - theta))

    # re-compute per-state reward and return
    return features.dot(theta)


# -- maximum causal entropy (Ziebart 2010) -------------------------------------

def softmax(x1, x2):
    """
    Computes a soft maximum of both arguments.

    In case `x1` and `x2` are arrays, computes the element-wise softmax.

    Args:
        x1: Scalar or ndarray.
        x2: Scalar or ndarray.

    Returns:
        The soft maximum of the given arguments, either scalar or ndarray,
        depending on the input.
    """
    x_max = np.maximum(x1, x2)
    x_min = np.minimum(x1, x2)
    return x_max + np.log(1.0 + np.exp(x_min - x_max))


def local_causal_action_probabilities(p_transition, terminal, reward, discount, eps=1e-5):
    """
    Compute the local action probabilities (policy) required for the edge
    frequency calculation for maximum causal entropy reinfocement learning.

    This is Algorithm 9.1 from Ziebart's thesis (2010) combined with
    discounting for convergence reasons as proposed in the same thesis.

    Args:
        p_transition: The transition probabilities of the MDP as table
            `[from: Integer, to: Integer, action: Integer] -> probability: Float`
            specifying the probability of a transition from state `from` to
            state `to` via action `action` to succeed.
        terminal: Either the terminal reward function or a collection of
            terminal states. Iff `len(terminal)` is equal to the number of
            states, it is assumed to contain the terminal reward function
            (phi) as specified in Ziebart's thesis. Otherwise `terminal` is
            assumed to be a collection of terminal states from which the
            terminal reward function will be derived.
        reward: The reward signal per state as table
            `[state: Integer] -> reward: Float`.
        discount: A discounting factor as Float.
        eps: The threshold to be used as convergence criterion for the state
            partition function. Convergence is assumed if the state
            partition function changes less than the threshold on all states
            in a single iteration.

    Returns:
        The local action probabilities (policy) as map
        `[state: Integer, action: Integer] -> probability: Float`
    """
    n_states, _, n_actions = p_transition.shape

    # set up terminal reward function
    if len(terminal) == n_states:
        reward_terminal = np.array(terminal, dtype=np.float)
    else:
        reward_terminal = -np.inf * np.ones(n_states)
        reward_terminal[terminal] = 0.0

    # set up transition probability matrices
    p = [np.array(p_transition[:, :, a]) for a in range(n_actions)]

    # compute state log partition V and state-action log partition Q
    v = -1e200 * np.ones(n_states)  # np.dot doesn't behave with -np.inf

    delta = np.inf
    while delta > eps:
        v_old = v

        q = np.array([reward + discount * p[a].dot(v_old) for a in range(n_actions)]).T

        v = reward_terminal
        for a in range(n_actions):
            v = softmax(v, q[:, a])

        # for some reason numpy chooses an array of objects after reduction, force floats here
        v = np.array(v, dtype=float)

        delta = np.max(np.abs(v - v_old))

    # compute and return policy
    return np.exp(q - v[:, None])


def compute_expected_causal_svf(p_transition, p_initial, terminal, reward, discount,
                                eps_lap=1e-5, eps_svf=1e-5):
    """
    Compute the expected state visitation frequency for maximum causal
    entropy IRL.

    This is a combination of Algorithm 9.1 and 9.3 of Ziebart's thesis
    (2010). See `local_causal_action_probabilities` and
    `expected_svf_from_policy` for more details.

    Args:
        p_transition: The transition probabilities of the MDP as table
            `[from: Integer, to: Integer, action: Integer] -> probability: Float`
            specifying the probability of a transition from state `from` to
            state `to` via action `action` to succeed.
        p_initial: The probability of a state being an initial state as map
            `[state: Integer] -> probability: Float`.
        terminal: Either the terminal reward function or a collection of
            terminal states. Iff `len(terminal)` is equal to the number of
            states, it is assumed to contain the terminal reward function
            (phi) as specified in Ziebart's thesis. Otherwise `terminal` is
            assumed to be a collection of terminal states from which the
            terminal reward function will be derived.
        reward: The reward signal per state as table
            `[state: Integer] -> reward: Float`.
        discount: A discounting factor as Float.
        eps_lap: The threshold to be used as convergence criterion for the
            state partition function. Convergence is assumed if the state
            partition function changes less than the threshold on all states
            in a single iteration.
        eps_svf: The threshold to be used as convergence criterion for the
            expected state-visitation frequency. Convergence is assumed if
            the expected state visitation frequency changes less than the
            threshold on all states in a single iteration.
    """
    p_action = local_causal_action_probabilities(p_transition, terminal, reward, discount, eps_lap)
    return expected_svf_from_policy(p_transition, p_initial, terminal, p_action, eps_svf)


def irl_causal(p_transition, features, terminal, trajectories, optim, init, discount,
               eps=1e-4, eps_svf=1e-5, eps_lap=1e-5):
    """
    Compute the reward signal given the demonstration trajectories using the
    maximum causal entropy inverse reinforcement learning algorithm proposed
    Ziebart's thesis (2010).

    Args:
        p_transition: The transition probabilities of the MDP as table
            `[from: Integer, to: Integer, action: Integer] -> probability: Float`
            specifying the probability of a transition from state `from` to
            state `to` via action `action` to succeed.
        features: The feature-matrix (e.g. as numpy array), mapping states
            to features, i.e. a matrix of shape (n_states x n_features).
        terminal: Either the terminal reward function or a collection of
            terminal states. Iff `len(terminal)` is equal to the number of
            states, it is assumed to contain the terminal reward function
            (phi) as specified in Ziebart's thesis. Otherwise `terminal` is
            assumed to be a collection of terminal states from which the
            terminal reward function will be derived.
        trajectories: A list of `Trajectory` instances representing the
            expert demonstrations.
        optim: The `Optimizer` instance to use for gradient-based
            optimization.
        init: The `Initializer` to use for initialization of the reward
            function parameters.
        discount: A discounting factor for the log partition functions as
            Float.
        eps: The threshold to be used as convergence criterion for the
            reward parameters. Convergence is assumed if all changes in the
            scalar parameters are less than the threshold in a single
            iteration.
        eps_lap: The threshold to be used as convergence criterion for the
            state partition function. Convergence is assumed if the state
            partition function changes less than the threshold on all states
            in a single iteration.
        eps_svf: The threshold to be used as convergence criterion for the
            expected state-visitation frequency. Convergence is assumed if
            the expected state visitation frequency changes less than the
            threshold on all states in a single iteration.
    """
    n_states, _, n_actions = p_transition.shape
    _, n_features = features.shape

    # compute static properties from trajectories
    e_features = feature_expectation_from_trajectories(features, trajectories)
    p_initial = initial_probabilities_from_trajectories(n_states, trajectories)

    # basic gradient descent
    theta = init(n_features)
    delta = np.inf

    optim.reset(theta)
    rewards = [features.dot(theta)]
    epoch = 0
    while delta > eps and epoch < 10000:
        theta_old = theta.copy()

        # compute per-state reward
        reward = features.dot(theta)

        # compute the gradient
        e_svf = compute_expected_causal_svf(p_transition, p_initial, terminal, reward, discount,
                                            eps_lap, eps_svf)

        grad = e_features - features.T.dot(e_svf)

        # perform optimization step and compute delta for convergence
        optim.step(grad)
        delta = np.max(np.abs(theta_old - theta))
        rewards.append(features.dot(theta))

        epoch += 1
    print(epoch, delta)

    # re-compute per-state reward and return
    return rewards


In [ ]:
X_df = X_df.drop(['b_cluster', 'k_cluster'], axis=1)

In [ ]:
import json
def perform_alg(smoothing_value, discount, cluster_algo='minibatchK'):

  p_transition = np.zeros((num_clusters, num_clusters, 15)) + smoothing_value

  for traj in trajectories:
    for tran in traj._t:
      p_transition[tran[0], tran[2], tran[1]] +=1

  p_transition = p_transition/ p_transition.sum(axis = 1)[:, np.newaxis, :]

  state_encoder = OneHotEncoder(sparse=False, categories= [np.arange(num_clusters)])
  action_encoder = OneHotEncoder(sparse=False, categories= [np.arange(15)])


  states_onehot = state_encoder.fit_transform(X_df['m_cluster'].to_numpy().reshape(-1, 1))
  actions_onehot = action_encoder.fit_transform(y_df.to_numpy().reshape(-1, 1))

  # set up features: we use one feature vector per state (1 hot encoding for each cluster/state)
  features = state_encoder.transform(np.arange(num_clusters).reshape(-1, 1))

  # choose our parameter initialization strategy:
  #   initialize parameters with constant
  init = O.Constant(1.0)

  # choose our optimization strategy:
  #   we select exponentiated stochastic gradient descent with linear learning-rate decay
  optim = O.ExpSga(lr=O.linear_decay(lr0=0.07))

  # actually do some inverse reinforcement learning
  # reward_maxent = maxent_irl(p_transition, features, terminal_states, trajectories, optim, init, eps= 1e-3)

  reward_maxent_causal = irl_causal(p_transition, features, terminal_states, trajectories, optim, init, discount,
                eps=1e-3, eps_svf=1e-4, eps_lap=1e-4)
  for i in range(len(reward_maxent_causal)):
    reward_maxent_causal[i] = reward_maxent_causal[i].tolist()

  file_name = 'mce_' + cluster_algo + str(discount)+'_smooth'+str(smoothing_value)+'.json'
  with open(file_name, 'w') as m:
      json.dump(reward_maxent_causal, m)


In [ ]:
for i in [0.000001, 0.3, 1]:
  for j in [0.001, 0.01, 0.3, 0.9]:
    perform_alg(i, j)
    print('finished', i, j)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


4343 0.0009999197878336474
finished 1e-06 0.001


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


4343 0.0009999197878336474
finished 1e-06 0.01


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


...

4343 0.0009999197878336474
finished 1 0.01


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


4343 0.0009999197878336474
finished 1 0.3


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


4343 0.0009999197878336474
finished 1 0.9
